In [1]:
!pip install qiskit matplotlib qiskit_aer qiskit_ibm_runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0

# Lab 1

In [1]:
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# transpiler passes
from qiskit.transpiler.passes.layout import csp_layout, dense_layout, sabre_layout, vf2_layout, trivial_layout
# Routing pa″sses
from qiskit.transpiler.passes.routing import basic_swap, lookahead_swap, sabre_swap
from qiskit.transpiler.passes.routing import star_prerouting
# Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis import basis_translator
from qiskit.transpiler.passes.synthesis import unitary_synthesis, high_level_synthesis
# Optimization passes
from qiskit.transpiler.passes.optimization import commutative_cancellation, consolidate_blocks, collect_2q_blocks, collect_1q_runs

In [2]:
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [3]:
def get_qc_chars(qc):
  depth = qc.depth
  num_qubits = qc.num_qubits
  ops = dict(qc.count_ops())
  num_multi_qubit_ops = qc.num_nonlocal_gates()

  return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}

In [4]:
def print_qc_characteristics(qc):
    characteristics = get_qc_chars(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

In [5]:
print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: <bound method BlueprintCircuit.depth of <qiskit.circuit.library.basis_change.qft.QFT object at 0x1434e5b90>>
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [6]:
qc_dec = qc.decompose()
print_qc_characteristics(qc_dec)

Quantum circuit characteristics
  Depth: <bound method QuantumCircuit.depth of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1451ebb10>>
  Number of qubits: 10
  Operations: {'cp': 45, 'h': 10}
  Number of multi-qubit Operations: 45


In [7]:
backend = GenericBackendV2(num_qubits)
print("Supported operations by the backend:", backend.operation_names)

Supported operations by the backend: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [8]:
qc_synth = generate_preset_pass_manager(3, backend, seed_transpiler=307).run(qc)
qc_synth.draw(fold=-1)

global phase: 1.5723
         ┌─────────┐┌────┐┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
q_9 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(3.1385) ├──■────────────────■────────────────■──────────────────────────────■───────────────■─────────────────────────────■────────────────■──────────────────────────────────■───────────────────────■───────────────────────────────────■─────────────────────■─────────────────────────────────────────■──────────────────────■──────────────────────────────────────────────■──────────────────────■───────────────────────────────────────────────■───────────────────────────■─────────────────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         └─────────┘└────┘└────────────┘┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐  │     ┌────┐   ┌────────────┐  │               │                             │                │                                  │                       │                                   │                     │                                         │                      │                                              │                      │                                               │                           │                                                     │                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
q_8 -> 1 ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├──┼─────┤ √X ├───┤ Rz(3.1355) ├──┼───────────────┼────────■────────────────────┼───────■────────┼─────────────────■────────────────┼──────────────────■────┼─────────────────■─────────────────┼────────────────■────┼──────────────────■──────────────────────┼─────────────────■────┼───────────────────────■──────────────────────┼─────────────────■────┼───────────────────────■───────────────────────┼──────────────────────■────┼────────────

In [9]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: <bound method QuantumCircuit.depth of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1575605d0>>
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [10]:
cm = CouplingMap.from_line(num_qubits)

backend = GenericBackendV2(num_qubits, coupling_map=cm)
qc_routed = generate_preset_pass_manager(3, backend).run(qc)
qc_routed.draw(fold=-1)

global phase: 0.001534
                                                                                                                                  ┌───┐┌───────────┐┌───┐┌──────────┐                            ┌───┐┌──────────┐┌───┐┌─────────┐                                   ┌───┐    ┌──────────┐   ┌───┐     ┌──────────┐┌────┐┌─────────┐                                                                                                                                                                                                                                                                               ┌───┐                                                                        ┌───┐     ┌──────────┐    ┌───┐     ┌─────────┐                                          ┌───┐     ┌──────────┐    ┌───┐     ┌──────────┐    ┌────┐    ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           ┌───┐                                                                     ┌───┐                                                                                                                                                            
q_6 -> 0 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/16) ├┤ X ├┤ Rz(π/16) ├────────────────────────────┤ X ├┤ Rz(-π/8) ├┤ X ├┤ Rz(π/8) ├───────────────────────────────────┤ X ├────┤ Rz(-π/4) ├───┤ X ├─────┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────┤ X ├─────────■──────────────────────────────────────────────────────────────┤ X ├─────┤ Rz(-π/8) ├────┤ X ├─────┤ Rz(π/8) ├──────────────────────────────────────────┤ X ├─────┤ Rz(-π/4) ├────┤ X ├─────┤ Rz(3π/4) ├────┤ √X ├────┤ Rz(15π/16) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────■──────┤ X ├──────■───────────────────────────────────────────────────────■──────┤ X ├──■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        ┌───┐┌──────────┐┌───┐┌──────────┐   ┌────┐   ┌─────────┐           ┌───┐     ┌──────────┐└─┬─┘└───────────┘└─┬─┘└──────────┘┌───┐            ┌─────────┐└─┬─┘└──────────┘└─┬─┘└─────────┘┌───┐               ┌─────────┐    └─┬─┘    └──────────┘   └─┬─┘     └──────────┘└────┘└─────────┘                                                                                                                                 ┌───┐                                                                         ┌───┐     ┌───────────┐    ┌───┐     ┌──────────┐    ┌─┴─┐      └─┬─┘       ┌─┴─┐     ┌──────────┐                                           └─┬─┘     └──────